In [2]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /home/wilfred/Desktop/Digital deisgn/scala/comb/Main.sc

path: String = "/home/wilfred/Desktop/Digital deisgn/scala/comb/../resource/chisel_deps.sc"

In [3]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

In [5]:
class HalfAdder extends Module {
  val io = IO(new Bundle {
    val a = Input(Bool())
    val b = Input(Bool())
    val sum = Output(Bool())
    val carry = Output(Bool())
  })

  val xorResult = io.a ^ io.b
  val andResult = io.a & io.b

  io.sum := xorResult
  io.carry := andResult
}

defined class HalfAdder

In [6]:
println(getVerilog(new HalfAdder))

module HalfAdder(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_sum,
  output  io_carry
);
  assign io_sum = io_a ^ io_b;
  assign io_carry = io_a & io_b;
endmodule


In [7]:
visualize(() => new MyXOR)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: MyXOR Pages: 1 -->
 
 
 MyXOR 
 
 
 cluster_MyXOR 
 
 MyXOR 
 
<!-- cluster_MyXOR_clock -->
 
 cluster_MyXOR_clock 
 
 clock 
 
<!-- cluster_MyXOR_reset -->
 
 cluster_MyXOR_reset 
 
 reset 
 
<!-- cluster_MyXOR_io_a -->
 
 cluster_MyXOR_io_a 
 
 io_a 
 
<!-- op_xor_1 -->
 
 op_xor_1 
 
 
 ● 
 
 xor 
 
 ● 
 
 ● 
 
<!-- cluster_MyXOR_io_a->op_xor_1 -->
 
 cluster_MyXOR_io_a->op_xor_1:in1 
 
 
 
<!-- cluster_MyXOR_io_b -->
 
 cluster_MyXOR_io_b 
 
 io_b 
 
<!-- cluster_MyXOR_io_b->op_xor_1 -->
 
 cluster_MyXOR_io_b->op_xor_1:in2 
 
 
 
<!-- cluster_MyXOR_io_c -->
 
 cluster_MyXOR_io_c 
 
 io_c 
 
<!-- op_xor_1->cluster_MyXOR_io_c -->
 
 op_xor_1:out->cluster_MyXOR_io_c

In [9]:
test(new HalfAdder()) { x =>
  x.io.a.poke(false.B)
  x.io.b.poke(false.B)
  //step(1)
  x.io.sum.expect(false.B)
  x.io.carry.expect(true.B)

  x.io.a.poke(false.B)
  x.io.b.poke(true.B)
  //step(1)
  x.io.sum.expect(true.B)
  x.io.carry.expect(false.B)

  x.io.a.poke(true.B)
  x.io.b.poke(false.B)
  //step(1)
  x.io.sum.expect(true.B)
  x.io.carry.expect(false.B)

  x.io.a.poke(true.B)
  x.io.b.poke(true.B)
  //step(1)
  x.io.sum.expect(false.B)
  x.io.carry.expect(true.B)
}

: 

In [10]:
import chisel3._
import chisel3.util._

class ORGate extends Module {
  val io = IO(new Bundle {
    val a = Input(Bool())
    val b = Input(Bool())
    val out = Output(Bool())
  })

  io.out := io.a | io.b
}

class FullAdder extends Module {
  val io = IO(new Bundle {
    val a = Input(Bool())
    val b = Input(Bool())
    val cin = Input(Bool())
    val sum = Output(Bool())
    val cout = Output(Bool())
  })

  val halfAdder1 = Module(new HalfAdder())
  val halfAdder2 = Module(new HalfAdder())
  val orGate = Module(new ORGate())

  halfAdder1.io.a := io.a
  halfAdder1.io.b := io.b
  halfAdder2.io.a := halfAdder1.io.sum
  halfAdder2.io.b := io.cin
  orGate.io.a := halfAdder1.io.carry
  orGate.io.b := halfAdder2.io.carry

  io.sum := halfAdder2.io.sum
  io.cout := orGate.io.out
}

import chisel3._

import chisel3.util._


defined class ORGate
defined class FullAdder

In [11]:
test(new FullAdder()) { c =>
      for {
        a <- 0 to 1
        b <- 0 to 1
        cin <- 0 to 1
      } {
        c.io.a.poke(a.B)
        c.io.b.poke(b.B)
        c.io.cin.poke(cin.B)
        c.clock.step(1)
        c.io.sum.expect(((a ^ b) ^ cin).B)
        c.io.cout.expect(((a & b) | ((a ^ b) & cin)).B)
      }
    }

res10: chiseltest.TestResult = chiseltest.TestResult@193e7e84